In [0]:
df = spark.read.format("parquet").load("abfss://source@sadec25etlproject.dfs.core.windows.net/orders")
df.display()

# **Data Reading**

In [0]:
df = spark.readStream.format("cloudfiles")\
    .option("cloudFiles.format", "parquet")\
    .option("cloudFiles.schemaLocation", "abfss://bronze@sadec25etlproject.dfs.core.windows.net/checkpoint_orders")\
    .load("abfss://source@sadec25etlproject.dfs.core.windows.net/orders")
#rerun this command everytime when the new data is added to the orders folder

## **Data Writing**

In [0]:
df.writeStream.format("parquet")\
    .outputMode("append")\
    .option("checkpointLocation", "abfss://bronze@sadec25etlproject.dfs.core.windows.net/checkpoint_orders")\
    .option("path", "abfss://bronze@sadec25etlproject.dfs.core.windows.net/orders")\
    .trigger(once=True)\
    .start()
#rerun this command everytime when the new data is added to the orders folder, and it will only append the new data because 
#  of trigger once, rocksdb

In [0]:
df = spark.read.format("parquet").load("abfss://bronze@sadec25etlproject.dfs.core.windows.net/orders")
df.display()

# **NEXT LEVEL**
## **Dynamic Capabilities**

In [0]:
dbutils.widgets.text("file_name", "orders")

In [0]:
dbutils.widgets.get("file_name")

In [0]:
p_file_name = dbutils.widgets.get("file_name")


# **Data Reading**

In [0]:
df = spark.readStream.format("cloudfiles")\
    .option("cloudFiles.format", "parquet")\
    .option("cloudFiles.schemaLocation", f"abfss://bronze@sadec25etlproject.dfs.core.windows.net/checkpoint_{p_file_name}")\
    .load(f"abfss://source@sadec25etlproject.dfs.core.windows.net/{p_file_name}")


## **Data Writing**

In [0]:
df.writeStream.format("parquet")\
    .outputMode("append")\
    .option("checkpointLocation", f"abfss://bronze@sadec25etlproject.dfs.core.windows.net/checkpoint_{p_file_name}")\
    .option("path", f"abfss://bronze@sadec25etlproject.dfs.core.windows.net/{p_file_name}")\
    .trigger(once=True)\
    .start()


In [0]:
# cheking the results by passing the file_name from parameter widget box after task success in jobs
df = spark.read.format("parquet")\
    .load(f"abfss://bronze@sadec25etlproject.dfs.core.windows.net/{p_file_name}")
df.display()